In [1]:
%load_ext autoreload
%autoreload 2

# Exercise 1

<img src="./images/01.png" width=800>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from utils import train_network, View, set_seed
import mlflow
from torchinfo import summary
import os
from collections import defaultdict

/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_07/utils.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
os.environ['MLFLOW_TRACKING_URI'] = './mlruns07_1'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [7]:
mlflow.set_experiment('Exercise07_1')

<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_07/mlruns07_1/143507330168611334', creation_time=1750415411076, experiment_id='143507330168611334', last_update_time=1750415411076, lifecycle_stage='active', name='Exercise07_1', tags={}>

In [3]:
torch.backends.cudnn.deterministic = True
set_seed(42)

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

## Dataset and DataLoader

In [5]:
# Train dataset
class AutoencodTrainDataset(Dataset):
    def __init__(self, dataset, label_excuded=[5, 9]):
        super().__init__()
        self.dataset = dataset
        self.valid_indexes = []
        for i, (_, y) in enumerate(dataset):
            if y not in label_excuded:
                self.valid_indexes.append(i)
        
    def __len__(self):
        return len(self.valid_indexes)
    def __getitem__(self, index):
        original_index = self.valid_indexes[index]
        x, y = self.dataset[original_index]
        return  x, y

In [11]:
class AutoencodTestDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.dataset = dataset
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, index):
        # x, y = self.dataset.__getitem__(index)
        x, y = self.dataset[index]
        return  x, x

In [ ]:
train_data = AutoencodTrainDataset(torchvision.datasets.MNIST("./data", train=True, transform=transforms.ToTensor(), download=True))
test_data_xy = torchvision.datasets.MNIST("./data", train=False, transform=transforms.ToTensor(), download=True)
test_data_xx = AutoencodTestDataset(test_data_xy)
batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data_xx, batch_size=batch_size)

## Model

In [ ]:
D = 28 * 28
n = 8
C =1
classes = 10

In [15]:
def getLayer(in_size, out_size):
    return nn.Sequential(
        nn.Linear(in_size, out_size),
        nn.BatchNorm1d(out_size),
        nn.ReLU()
    )

In [ ]:
auto_encoder = nn.Sequential(
    nn.Flatten(),
    getLayer(D, D//2),
    getLayer(D//2, D//3),
    nn.Linear(D//3, n),
)

auto_decoder = nn.Sequential(
    getLayer(n, D//3),
    getLayer(D//3, D//2),
    nn.Linear(D//2, D),
    nn.Sigmoid()
    View(-1, 1, 28, 28)
)

auto_encod_decode = nn.Sequential(
    auto_encoder,
    auto_decoder,
)

## Training

In [ ]:
loss_func = nn.MSELoss()
epochs = 50
optimizer = optim.AdamW(auto_encod_decode.parameters())
params = {
    'device': device,
    'loss_func': loss_func.__class__.__name__,
    'epochs': epochs,
    'batch_size': batch_size,
    'optimizer': optimizer.defaults
    }

In [ ]:
with open('model_summary.txt', 'w') as f:
    f.write(str(summary(auto_encod_decode, inpt_size=(batch_size, C, 28, 28))))
with mlflow.start_run(nested=True, run_name='training without 9 and 5'):
    mlflow.log_artifact('model_summary.txt')
    mlflow.log_params(params)

    results = train_network(
        model=auto_encod_decode,
        optimizer=optimizer,
        loss_func=loss_func,
        train_loader=train_loader,
        valid_loader=test_loader,
        epochs=epochs,
        device=device,
        checkpoint_file_save='model.pth',
        
    )

<img src="./images/E1_train_loss.png">

<img src="./images/E1_valid_loss.png">

<img src="./images/E1_time.png">

#

## Results

In [ ]:
def load_model_from_mlflow(
    run_id, artifact_path, model, device
    ):
    artifact_uri = f'runs:/{run_id}/{artifact_path}'
    checkpoint_path = mlflow.artifacts.download_artifacts(artifact_uri=artifact_uri)
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer_state_dict = checkpoint['optimizer_state_dict']
    results = checkpoint['results']
    epoch = checkpoint['epoch']
    model.eval()
    model.to(device)
    return model, results, epoch

In [ ]:
run_id = 'b91804fffef54b109edf4cd385681ae9'
artifact_path = 'model.pth'
model, *_ = load_model_from_mlflow(
        run_id=run_id,
        artifact_path=artifact_path,
        model=auto_encod_decode, 
        device=device
    )

In [ ]:
mse_for_labels = defaultdict(float)
model = model.to('cpu')
model = model.eval()
with torch.no_grad():
    for idx, (x, y) in enumerate(test_data_xy):
        if not idx%100:
            print(idx)
        x_reco = model(x)
        mse_for_labels[y] +=loss_func(x_reco, x)

<img src='./images/E1_mse.png'>